# __Toronto Neighborhood__

## Part 1

In [1]:
!pip install bs4
!pip install lxml

In [42]:
import pandas as pd
import requests
from lxml import html
from bs4 import BeautifulSoup

In [43]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content, "lxml")
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df=pd.DataFrame(df[0])

  
  
Deleting the boroughs that are not assigned

In [44]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)




Current Postal code list on Wikipedia page is already rouped.  There is no need to set a neighborhood name as the same as the borough. We have to replace the slash with the comma

In [45]:
df = df.replace({'/': ','}, regex=True)

In [64]:
PCdf=df.sort_values(by=['Postal code']).rename(columns={'Postal code': 'Postal Code'})
PCdf.reset_index(inplace=True)
PCdf.drop('index',axis=1,inplace=True)

In [65]:
PCdf

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [66]:
print(PCdf.shape)

(103, 3)



-----------------------------------------------------------------------
## Part 2

In [67]:
coord=pd.read_csv('Geospatial_Coordinates.csv') 

In [69]:
PCdf.merge(coord,on='Postal Code')

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
